# Involves cleaning of text messages, date column and from column

In [1]:
import pandas as pd
import re
import matplotlib as plt

df=pd.read_csv('new2.csv')
df.head()

,id,date,text,from,from_id,reply_to_message_id,contact_information,contact_vcard
0,9799,2020-06-10T22:58:32,Make a random model and calc the rmse score th...,Dragon Master,1217107241,9796.0,NaN,NaN
1,9804,2020-06-10T23:05:42,Hi all,Rohit Jain,1250027868,NaN,NaN,NaN
2,9805,2020-06-10T23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain,1250027868,NaN,NaN,NaN
3,9809,2020-06-10T23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma,1117859453,NaN,NaN,NaN
4,9811,2020-06-10T23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma,1117859453,NaN,NaN,NaN


Converting text column to string datatype and show some rows containing text in diffrent types and will be cleaned later 

In [2]:
df['text'] = df['text'].astype(str)

In [3]:
df[df['id']==9886][['text']]

,text
56,But I am more comfortable with sql


In [4]:
df[df['id']==9852][['text']]

,text
27,"[{'type': 'mention_name', 'text': 'Mohak', 'us..."


In [5]:
df[df['id']==9895][['text']]

,text
61,"['Any please help. ', {'type': 'bold', 'text'..."


Function to clean the contents of text messages and store them to message column

In [6]:
def message_clean(df):
    message = ''
    if df.loc['text'][0] == '[':
        text = df.loc['text'].strip().strip('[').strip(']')        
        escapes = ''.join([chr(char) for char in range(1, 32)])
        translator = str.maketrans('', '', escapes)
        t = text.translate(translator)
        results = re.findall(r"'(.*?)'|\"(.*?)\"|({.*?})", t) #check for text only and dictionary using regex
        results = [item for elem in results for item in elem if len(item)] # Clean empty records
        for e in results:
            if(e[0]=='{'):
                d = eval(e)
                message = message + " " + d['text'].strip()
            else:
                message = message + " " + e.strip()
    else:
        message = df.loc['text']
    return message

In [7]:
df['message'] = df.apply(message_clean,axis=1)
df.head()

,id,date,text,from,from_id,reply_to_message_id,contact_information,contact_vcard,message
0,9799,2020-06-10T22:58:32,Make a random model and calc the rmse score th...,Dragon Master,1217107241,9796.0,NaN,NaN,Make a random model and calc the rmse score th...
1,9804,2020-06-10T23:05:42,Hi all,Rohit Jain,1250027868,NaN,NaN,NaN,Hi all
2,9805,2020-06-10T23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain,1250027868,NaN,NaN,NaN,Ha my name is rohit jain and I joined your gro...
3,9809,2020-06-10T23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma,1117859453,NaN,NaN,NaN,I'm working on a dataset where text and it's s...
4,9811,2020-06-10T23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma,1117859453,NaN,NaN,NaN,Can anyone in the group help me out... I'm ha...


Drop Nan values in text column

In [8]:
df.dropna(subset=['text'],inplace=True)

Row where contact was shared and dropping them

In [9]:
df[df['id']==25317]

,id,date,text,from,from_id,reply_to_message_id,contact_information,contact_vcard,message
9752,25317,2020-07-18T13:49:26,nan,sartaj,677791092,NaN,"{'first_name': 'Anupam', 'last_name': '', 'pho...",contacts/contact_1.vcard,nan


In [10]:
df.drop(df[df.text == 'nan'].index , inplace=True)
df.drop(df[df.text == 'NaN'].index , inplace=True)

In [11]:
df.columns

Index(['id', 'date', 'text', 'from', 'from_id', 'reply_to_message_id',
       'contact_information', 'contact_vcard', 'message'],
      dtype='object')

In [12]:
df = df[['id', 'date', 'text', 'from', 'from_id', 'reply_to_message_id','message']]

Cleaning date column and converting it into datetime datatype

In [13]:
df['date'] = df.apply(lambda x: ' '.join(x['date'].split('T')),axis=1).astype('datetime64[ns]')

Concat columns from and from_id

In [14]:
df['from'] = df['from'] + ' ' + '( ' + df['from_id'].astype(str) + ' )'

In [15]:
df.dtypes

id                              int64
date                   datetime64[ns]
text                           object
from                           object
from_id                         int64
reply_to_message_id           float64
message                        object
dtype: object

In [16]:
df.head()

,id,date,text,from,from_id,reply_to_message_id,message
0,9799,2020-06-10 22:58:32,Make a random model and calc the rmse score th...,Dragon Master ( 1217107241 ),1217107241,9796.0,Make a random model and calc the rmse score th...
1,9804,2020-06-10 23:05:42,Hi all,Rohit Jain ( 1250027868 ),1250027868,NaN,Hi all
2,9805,2020-06-10 23:07:36,Ha my name is rohit jain and I joined your gro...,Rohit Jain ( 1250027868 ),1250027868,NaN,Ha my name is rohit jain and I joined your gro...
3,9809,2020-06-10 23:18:52,I'm working on a dataset where text and it's s...,Mukul Sharma ( 1117859453 ),1117859453,NaN,I'm working on a dataset where text and it's s...
4,9811,2020-06-10 23:19:46,Can anyone in the group help me out... I'm ha...,Mukul Sharma ( 1117859453 ),1117859453,NaN,Can anyone in the group help me out... I'm ha...


Keeping required columns and making a final cleaned .csv file

In [17]:
final = df[['id','date','from','message','reply_to_message_id']]

In [18]:
final.head()

,id,date,from,message,reply_to_message_id
0,9799,2020-06-10 22:58:32,Dragon Master ( 1217107241 ),Make a random model and calc the rmse score th...,9796.0
1,9804,2020-06-10 23:05:42,Rohit Jain ( 1250027868 ),Hi all,NaN
2,9805,2020-06-10 23:07:36,Rohit Jain ( 1250027868 ),Ha my name is rohit jain and I joined your gro...,NaN
3,9809,2020-06-10 23:18:52,Mukul Sharma ( 1117859453 ),I'm working on a dataset where text and it's s...,NaN
4,9811,2020-06-10 23:19:46,Mukul Sharma ( 1117859453 ),Can anyone in the group help me out... I'm ha...,NaN


In [19]:
final.to_csv('finalchat.csv',index=False)